In [1]:
import pandas as pd
import numpy as np
from scipy import stats

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM

gpu_devices = tf.config.experimental.list_physical_devices("GPU")
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [2]:
def create_data(X,Y):
    time_step=32;
    x_train, y_train = [], []
    x_test, y_test = [], []

    for i in range(0,Y.size-time_step,1):
        labels = Y.iloc[i:(i+time_step)]
        mode = stats.mode(labels)[0][0]
        if mode == 2 :
            y_train.append(1)
        if mode == 3 :
            y_train.append(0)
        if mode == 4:
            y_test.append(1)
        if mdoe == 5:
            y_test.append(0)
    
    for i in range(0,len(y_train)-time_step,1):
        tmp = X.iloc[i:(i+time_step)]
        x_train.append(tmp)
    for i in range(len(y_train),len(y_test)-time_step,1):
        tmp = X.iloc[i:(i+time_step)]
        x_test.append(tmp)
        
    return np.array(x_train), np.array(y_train).reshape(-1,1), np.array(x_test), np.array(y_test).reshape(-1,1)

In [ ]:
def data():
    df = pd.read_csv('../csv/gsr_record.csv',header=0)#change file name later
    df = df.drop(df[df.STATE==1].index)
    x_train, y_train, x_test, y_test = create_data(df['GSR'],df['STATE'])
    
    return x_train, y_train, x_test, y_test

In [3]:
def model(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(LSTM({{choice([8,16,32])}}, input_shape=(X_train.shape[1],1),return_sequences=True))
    model.add(Dropout({{uniform(0,1)}}))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.fit(x_train,
              y_train,
              epochs={{choice([16,32,64])}},
              batch_size={{choice([32,64,128])}},
              validation_data=(x_test,y_test),
              shuffle=False)
    val_loss, val_acc = model.evaluate(x_test, y_test, verbose=0)
    return {'loss': -val_acc, 'status': STATUS_OK, 'model': model}

In [4]:
if __name__ == "__main__":
    best_run, best_model = optim.minimize(model=model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials())

    print(best_model.summary())
    print(best_run)

    x_train, y_train, x_test, y_test = prepare_data()
    val_loss, val_acc = best_model.evaluate(x_test, y_test)
    print("val_loss: ", val_loss)
    print("val_acc: ", val_acc)

NameError: name 'data' is not defined